In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
# Read the datasets
hot100_df = pd.read_csv('../data/1_hot100.csv')
million_songs_df = pd.read_csv('../data/2_million_song_subset_raw.csv')

In [3]:
hot100_df.head()

,Rank,Title,Artist
0,1,Die With A Smile,Lady Gaga & Bruno Mars
1,2,A Bar Song (Tipsy),Shaboozey
2,3,Birds Of A Feather,Billie Eilish
3,4,Lose Control,RE-\nENTRY
4,5,APT.,ROSE & Bruno Mars


In [4]:
million_songs_df.head()

,title;artist
0,b'Je Sais Que La Terre Est Plate';b'Rapha\xc3\...
1,b'On Efface';b'Julie Zenatti'
2,b'Howells Delight';b'The Baltimore Consort'
3,b'Martha Served';b'I Hate Sally'
4,b'Zip-A-Dee-Doo-Dah (Song of the South)';b'Orl...


In [5]:
def clean_text(text):
    """Clean text by removing special characters and standardizing format"""
    if pd.isna(text):
        return text
    
    # Convert to string and handle byte strings
    text = str(text)
    if text.startswith("b'") or text.startswith('b"'):
        text = text[2:-1]  # Remove b'' or b""
    
    # Decode special characters
    text = text.encode('latin-1').decode('utf-8', errors='ignore')
    
    # Remove special characters and extra spaces
    text = re.sub(r'[^\w\s]', '', text)
    text = ' '.join(text.split())
    
    return text.lower().strip()

def clean_artist_name(artist):
    """Clean artist names by removing featuring, etc."""
    if pd.isna(artist):
        return artist
    
    artist = clean_text(artist)
    artist = re.sub(r'\([^)]*\)', '', artist)
    artist = re.sub(r'feat\..*|ft\..*', '', artist)
    
    return artist.strip()

def split_title_artist(row):
    """Split the combined title;artist string"""
    parts = str(row).split(';')
    return pd.Series({'Title': parts[0], 'Artist': parts[1] if len(parts) > 1 else ''})

In [6]:

# Clean Hot 100 dataset
hot100_clean = hot100_df.copy()
hot100_clean['Title'] = hot100_clean['Title'].apply(clean_text)
hot100_clean['Artist'] = hot100_clean['Artist'].apply(clean_artist_name)
hot100_clean['Hot100'] = True

# Display sample of cleaned Hot 100 data
print("Sample of cleaned Hot 100 data:")
hot100_clean.head()

Sample of cleaned Hot 100 data:


,Rank,Title,Artist,Hot100
0,1,die with a smile,lady gaga bruno mars,True
1,2,a bar song tipsy,shaboozey,True
2,3,birds of a feather,billie eilish,True
3,4,lose control,re entry,True
4,5,apt,rose bruno mars,True


In [7]:
# Split and clean Million Songs data
million_songs_clean = million_songs_df['title;artist'].apply(split_title_artist)
million_songs_clean['Title'] = million_songs_clean['Title'].apply(clean_text)
million_songs_clean['Artist'] = million_songs_clean['Artist'].apply(clean_artist_name)
million_songs_clean['Hot100'] = False


million_songs_clean.head()

,Title,Artist,Hot100
0,je sais que la terre est plate,raphaxc3xabl,False
1,on efface,julie zenatti,False
2,howells delight,the baltimore consort,False
3,martha served,i hate sally,False
4,zipadeedoodah song of the south,orlando pops orchestra,False


In [8]:
#Combine datasets
combined_df = pd.concat([
    hot100_clean[['Title', 'Artist', 'Hot100']],
    million_songs_clean[['Title', 'Artist', 'Hot100']]
], ignore_index=True)

# Remove duplicates
combined_df = combined_df.drop_duplicates(subset=['Title', 'Artist'])

# Check for missing values
print("\nMissing values in combined dataset:")
print(combined_df.isnull().sum())

# Basic statistics
print("\nDataset statistics:")
print(f"Total number of tracks: {len(combined_df)}")
print(f"Number of Hot 100 tracks: {combined_df['Hot100'].sum()}")
print(f"Number of Million Songs tracks: {(~combined_df['Hot100']).sum()}")


Missing values in combined dataset:
Title     0
Artist    0
Hot100    0
dtype: int64

Dataset statistics:
Total number of tracks: 10039
Number of Hot 100 tracks: 100
Number of Million Songs tracks: 9939


In [9]:
combined_df.to_csv('../data/3_combined_clean_dataset.csv', index=False)